In [1]:
import random
import os
import re
import json
import numpy as np
import pandas as pd
import lightgbm as lgbm
from datetime import datetime
from time import gmtime, strftime
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import torch
import lightgbm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error

from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from PIL import Image
# from PIL import ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True
# from transformers import ViltProcessor, ViltModel
from lavis.models import load_model_and_preprocess
from tqdm import trange


random_seed = 2020
random.seed(random_seed)
np.random.seed(random_seed)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-02 13:26:48.320935: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 13:26:49.365473: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:
2023-06-02 13:26:49.365598: W tensorflow/compiler/xla/stream_executor/platform/default/dso_lo

In [ ]:
# from transformers import ViltProcessor, ViltModel
# text = "a bunch of cats laying on a couch"
# # url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# # image = Image.open(requests.get(url, stream=True).raw)
# image = Image.open('../data/none_picture.jpg')
# processor = ViltProcessor.from_pretrained("../model/VilT_flickr")
# model = ViltModel.from_pretrained("../model/VilT_flickr")
# inputs = processor(image, text, return_tensors='pt')
# outputs = model(**inputs)
# # outputs.pooler_output

In [2]:
def get_img_data(img_file):
    if os.path.exists('../data/' + img_file) == True:
        return '../data/' + img_file
    else:
        return '../data/none_picture.jpg'

In [3]:
all_data = pd.read_csv('../data/combine_data_530.csv', low_memory=False, index_col=0)
all_data = all_data.fillna('0')
data_df = all_data
data_df

,Category,Concept,Pid,Uid,Subcategory,Mediastatus,Pathalias,Ispublic,Alltags,Mediatype,...,ispro,user_description,canbuypro,timezone_timezone_id,photo_firstdate,timezone_offset,img,img_file,label,train_type
0,Fashion,glam,775,59@N75,Fashion,ready,luisdrayton,1,rock punk transgender tranny electronicmusic e...,photo,...,1,"0.0866962,-0.0752717,-0.0149407,-0.0460858,-0....",0,9,1213743830,1,train/59@N75/775.jpg,train/59@N75/775.jpg,11.18,0
1,Travel&Active&Sports,fifa,1075,1@N18,Soccer,ready,claudiolara,1,brazil rio brasil riodejaneiro by maria fifa m...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1@N18/1075.jpg,train/1@N18/1075.jpg,15.15,0
2,Entertainment,cinema,4890,351@N64,Movies,ready,rizzato,1,old cinema beauty marilyn photoshop movie joke...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,11,1296417600,-3,train/351@N64/4890.jpg,train/351@N64/4890.jpg,10.99,0
3,Holiday&Celebrations,old,6568,6@N59,Birthday,ready,4kleuren,1,pictures old family scans brothers sister 1958...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/6@N59/6568.jpg,train/6@N59/6568.jpg,8.63,0
4,Food,thirsty,7079,1617@N40,Drinks,ready,glosackmd,1,hot sahara animal animals desert bottles drink...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1617@N40/7079.jpg,train/1617@N40/7079.jpg,11.16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,Travel&Active&Sports,sea,1118324,49686@N77,Seattle,ready,larsgerrits,1,natur nature frog sea lake fish outdoor macro ...,photo,...,1,"0.150263,0.00344591,-0.158953,0.118596,-0.0566...",0,8,1394046082,0,test/49686@N77/1118324.jpg,test/49686@N77/1118324.jpg,-1.00,-1
486190,Travel&Active&Sports,cars,1118329,31147@N75,Cars,ready,None,1,11 arisk okruh veternov 2016 10 jn preov veter...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1460319331,0,test/31147@N75/1118329.jpg,test/31147@N75/1118329.jpg,-1.00,-1
486191,Animal,cat,1118336,5656@N26,Cats,ready,rogersg,1,holland netherlands amsterdam cat nl ramses gu...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,9,1464117797,1,test/5656@N26/1118336.jpg,test/5656@N26/1118336.jpg,-1.00,-1
486192,Whether&Season,white,1118349,28302@N36,Snowing,ready,ripensis,1,street people bw white black netherlands monoc...,photo,...,1,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1246915883,0,test/28302@N36/1118349.jpg,test/28302@N36/1118349.jpg,-1.00,-1


In [4]:
import os
import torch
from PIL import Image
from lavis.models import load_model_and_preprocess
os.environ['CUDA_VISIBLE_DEVICES'] = '6,7'
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")
data_df['img_file'] = data_df['img_file'].apply(lambda x: get_img_data(x))

In [5]:
data_df

,Category,Concept,Pid,Uid,Subcategory,Mediastatus,Pathalias,Ispublic,Alltags,Mediatype,...,ispro,user_description,canbuypro,timezone_timezone_id,photo_firstdate,timezone_offset,img,img_file,label,train_type
0,Fashion,glam,775,59@N75,Fashion,ready,luisdrayton,1,rock punk transgender tranny electronicmusic e...,photo,...,1,"0.0866962,-0.0752717,-0.0149407,-0.0460858,-0....",0,9,1213743830,1,train/59@N75/775.jpg,../data/train/59@N75/775.jpg,11.18,0
1,Travel&Active&Sports,fifa,1075,1@N18,Soccer,ready,claudiolara,1,brazil rio brasil riodejaneiro by maria fifa m...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1@N18/1075.jpg,../data/train/1@N18/1075.jpg,15.15,0
2,Entertainment,cinema,4890,351@N64,Movies,ready,rizzato,1,old cinema beauty marilyn photoshop movie joke...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,11,1296417600,-3,train/351@N64/4890.jpg,../data/train/351@N64/4890.jpg,10.99,0
3,Holiday&Celebrations,old,6568,6@N59,Birthday,ready,4kleuren,1,pictures old family scans brothers sister 1958...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/6@N59/6568.jpg,../data/train/6@N59/6568.jpg,8.63,0
4,Food,thirsty,7079,1617@N40,Drinks,ready,glosackmd,1,hot sahara animal animals desert bottles drink...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1617@N40/7079.jpg,../data/train/1617@N40/7079.jpg,11.16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,Travel&Active&Sports,sea,1118324,49686@N77,Seattle,ready,larsgerrits,1,natur nature frog sea lake fish outdoor macro ...,photo,...,1,"0.150263,0.00344591,-0.158953,0.118596,-0.0566...",0,8,1394046082,0,test/49686@N77/1118324.jpg,../data/test/49686@N77/1118324.jpg,-1.00,-1
486190,Travel&Active&Sports,cars,1118329,31147@N75,Cars,ready,None,1,11 arisk okruh veternov 2016 10 jn preov veter...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1460319331,0,test/31147@N75/1118329.jpg,../data/test/31147@N75/1118329.jpg,-1.00,-1
486191,Animal,cat,1118336,5656@N26,Cats,ready,rogersg,1,holland netherlands amsterdam cat nl ramses gu...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,9,1464117797,1,test/5656@N26/1118336.jpg,../data/test/5656@N26/1118336.jpg,-1.00,-1
486192,Whether&Season,white,1118349,28302@N36,Snowing,ready,ripensis,1,street people bw white black netherlands monoc...,photo,...,1,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1246915883,0,test/28302@N36/1118349.jpg,../data/test/28302@N36/1118349.jpg,-1.00,-1


In [9]:
# data_df['Alltags'][0][:52]

'rock punk transgender tranny electronicmusic electro'

In [ ]:
from tqdm import trange
model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_feature_extractor", model_type="pretrain", is_eval=True, device=device)
with torch.no_grad():
    caption1 = data_df['Alltags'][0][:500]
    caption2 = data_df['Title'][0][:500]
    raw_image1 = Image.open(data_df['img_file'][0]).convert("RGB")
    image1 = vis_processors["eval"](raw_image1).unsqueeze(0).to(device)
    text_input1 = txt_processors["eval"](caption1)
    sample1 = {"image": image1, "text_input": [text_input1]}
    text_input2 = txt_processors["eval"](caption2)
    sample2 = {"image": image1, "text_input": [text_input2]}
    features_multimodal1 = model.extract_features(sample1).multimodal_embeds[0,0]
    features_multimodal2 = model.extract_features(sample2).multimodal_embeds[0,0]
    for index in trange(486193):
        try:
            raw_images = Image.open(data_df['img_file'][index + 1]).convert("RGB")
        except:
            raw_images = Image.open('../data/none_picture.jpg').convert("RGB")
        captions1 = data_df['Alltags'][index + 1][:500]
        captions2 = data_df['Title'][index + 1][:500]
        images = vis_processors["eval"](raw_images).unsqueeze(0).to(device)
        text_inputs1 = txt_processors["eval"](captions1)
        samples1 = {"image": images, "text_input": [text_inputs1]}
        text_inputs2 = txt_processors["eval"](captions2)
        samples2 = {"image": images, "text_input": [text_inputs2]}
        features_multimodals1 = model.extract_features(samples1).multimodal_embeds[0,0]
        features_multimodals2 = model.extract_features(samples2).multimodal_embeds[0,0]
        features_multimodal1 = torch.concat((features_multimodal1,features_multimodals1),0)
        features_multimodal2 = torch.concat((features_multimodal2,features_multimodals2),0)
    

 24%|███████████████████████████████████▏                                                                                                              | 117026/486193 [8:20:34<11:27:46,  8.95it/s]

In [ ]:
features_multimodal1_768 = pd.DataFrame(features_multimodal1.cpu().detach().numpy())
features_multimodal1_768.columns = ['blip_tagsfea_{}'.format(i) for i in range(768)]
features_multimodal1_768.to_csv('../data/blip_tagsfea_768.csv', header=True, index=None)

In [ ]:
features_multimodal2_768 = pd.DataFrame(features_multimodal2.cpu().detach().numpy())
features_multimodal2_768.columns = ['blip_titlefea_{}'.format(i) for i in range(768)]
features_multimodal2_768.to_csv('../data/blip_titlefea_768.csv', header=True, index=None)

In [ ]:







print(features_multimodal.multimodal_embeds.shape)

In [ ]:
data_df['img_file'] = data_df['img_file'].apply(lambda x: get_img_data(x))
with torch.no_grad():
# img_mode_map = {'P': 0, 'L': 1, 'RGB': 2, 'CMYK': 3, 'RGBA': 4}
    inputs1 = processor(Image.open(data_df['img_file'][0]), data_df['Title'][0] + ' ' + data_df['Alltags'][0], return_tensors='pt', padding=True , truncation=True).to(device)
    outputs1 = model(**inputs1)
    vilt_fea = outputs1.pooler_output
    for index in trange(486193):
        text = data_df['Title'][index + 1] + ', ' + data_df['Alltags'][index + 1]
        try:
            image = Image.open(data_df['img_file'][index + 1])  
            inputs = processor(image, text, return_tensors='pt', padding=True , truncation=True).to(device)
            outputs = model(**inputs)
            vilt_fea1 = outputs.pooler_output
            vilt_fea = torch.concat((vilt_fea,vilt_fea1),0)
        except: 
            image = Image.open('../data/none_picture.jpg')
            inputs = processor(image, text, return_tensors='pt', padding=True , truncation=True).to(device)
            outputs = model(**inputs)
            vilt_fea1 = outputs.pooler_output
            vilt_fea = torch.concat((vilt_fea,vilt_fea1),0)
vilt_fea_768 = pd.DataFrame(vilt_fea.cpu().detach().numpy())
vilt_fea_768.columns = ['viltvqa_fea_{}'.format(i) for i in range(768)]
vilt_fea_768.to_csv('../data/vilt_fea_vqa.csv', header=True, index=None)

In [ ]:
dataset_path = '/MMChallenge/SMP/data'
image_col = 'img_file'
def path_expander(path, base_folder):
    path_l = path.split(';')
    return ';'.join([os.path.abspath(os.path.join(base_folder, path)) for path in path_l])
data_df[image_col] = data_df[image_col].apply(lambda ele: path_expander(ele, base_folder=dataset_path))
# train_all_data[image_col] = train_all_data[image_col].apply(lambda ele: path_expander(ele, base_folder=dataset_path))
# submit_all_data[image_col] = submit_all_data[image_col].apply(lambda ele: path_expander(ele, base_folder=dataset_path))
# model, vis_processors, text_processors = load_model_and_preprocess(\"blip_image_text_matching\", \"large\", device=device, is_eval=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, vis_processors, text_processors = load_model_and_preprocess("blip_image_text_matching", "large", device=device, is_eval=True)

img = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
txt = text_processors["eval"](caption)
itc_score = model({"image": img, "text_input": txt}, match_head='itc')
# print('The image feature and text feature has a cosine similarity of %.4f'%itc_score)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_feature_extractor", model_type="pretrain", is_eval=True, device=device)
image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
text_input = txt_processors["eval"](caption)
sample = {"image": image, "text_input": [text_input]}
features_image = model.extract_features(sample, mode="image")
features_text = model.extract_features(sample, mode="text")
similarity = (features_image.image_embeds_proj @ features_text.text_embeds_proj[:,0,:].t()).max()
print(similarity)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_df['img_file'] = data_df['img_file'].apply(lambda x: get_img_data(x))
 # + ' ' + data_df['Alltags'][0]
model, vis_processors, text_processors = load_model_and_preprocess("blip_image_text_matching", "large", device=device, is_eval=True)
with torch.no_grad():
# img_mode_map = {'P': 0, 'L': 1, 'RGB': 2, 'CMYK': 3, 'RGBA': 4}
    input_Image1 = Image.open(data_df['img_file'][0]).convert("RGB")
    input_alltags1 = data_df['Alltags'][0]
    input_Title1 = data_df['Title'][0]
    # input_Image1 = data_df['img_file'][0]
    img1 = vis_processors["eval"](input_Image1).unsqueeze(0).to(device)
    alltags1 = text_processors["eval"](input_alltags1)
    Title1 = text_processors["eval"](input_Title1)
    
    itc_alltags1 = list(model({"image": img1, "text_input": alltags1}, match_head='itc'))
    itc_Title1 = list(model({"image": img1, "text_input": Title1}, match_head='itc'))
    
    for index in trange(486193):
        input_Image = Image.open(data_df['img_file'][index + 1]).convert("RGB")
        # input_Image = data_df['img_file'][index + 1]
        input_alltags = data_df['Alltags'][index + 1]
        input_Title = data_df['Title'][index + 1]
        img = vis_processors["eval"](input_Image).unsqueeze(0).to(device)
        alltags = text_processors["eval"](input_alltags)
        Title = text_processors["eval"](input_Title)
        
        itc_alltags = list(model({"image": img, "text_input": alltags}, match_head='itc'))
        itc_Title = list(model({"image": img, "text_input": Title}, match_head='itc'))
        itc_alltags1 += itc_alltags
        itc_Title1 += itc_Title
    
#     inputs1 = processor(Image.open(data_df['img_file'][0]), data_df['Title'][0], return_tensors='pt', padding=True , truncation=True).to(device)
#     outputs1 = model(**inputs1)
#     vilt_fea = outputs1.pooler_output
#     for index in trange(486193):
#         text = data_df['Title'][index + 1] + ', ' + data_df['Alltags'][index + 1]
#         try:
#             image = Image.open(data_df['img_file'][index + 1])  
#             inputs = processor(image, text, return_tensors='pt', padding=True , truncation=True).to(device)
#             outputs = model(**inputs)
#             vilt_fea1 = outputs.pooler_output
#             vilt_fea = torch.concat((vilt_fea,vilt_fea1),0)
#         except: 
#             image = Image.open('../data/none_picture.jpg')
#             inputs = processor(image, text, return_tensors='pt', padding=True , truncation=True).to(device)
#             outputs = model(**inputs)
#             vilt_fea1 = outputs.pooler_output
#             vilt_fea = torch.concat((vilt_fea,vilt_fea1),0)
# vilt_fea_768 = pd.DataFrame(vilt_fea.cpu().detach().numpy())
# vilt_fea_768.columns = ['viltvqa_fea_{}'.format(i) for i in range(768)]
# vilt_fea_768.to_csv('../data/vilt_fea_vqa.csv', header=True, index=None)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3.43G/3.43G [05:01<00:00, 12.2MB/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 486193/486193 [14:25:51<00:00,  9.36it/s]


In [12]:
# itc_alltags = itc_alltags1

In [21]:
itc_alltags = itc_alltags1
itc_alltags_CPU = [x.cpu().detach().item() for x in itc_alltags]
# itc_alltags_CPU

In [22]:
itc_Title = itc_Title1
itc_Title_CPU = [x.cpu().detach().item() for x in itc_Title]
# itc_alltags_CPU

In [29]:
itc_TAGS_CPU_df = pd.DataFrame(itc_alltags_CPU)
itc_TAGS_CPU_df.columns = ['Tags_simi']
itc_TAGS_CPU_df.to_csv('../data/alltags_simi.csv',header=True,index=None)

In [30]:
itc_Title_CPU_df = pd.DataFrame(itc_Title_CPU)
itc_Title_CPU_df.columns = ['Title_simi']
itc_Title_CPU_df.to_csv('../data/alltitle_simi.csv',header=True,index=None)

In [23]:
pd.DataFrame(itc_alltags_CPU).to_csv('../data/alltags_simi.csv',header=None,index=True)

In [ ]:
pd.DataFrame(itc_Title_CPU).to_csv('../data/allTitle_simi.csv',header=None,index=True)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = ViltProcessor.from_pretrained("../model/VilT_VQA", truncation = True, padding = True)
model = ViltModel.from_pretrained("../model/VilT_VQA")
model.to(device)

Some weights of the model checkpoint at ../model/VilT_VQA were not used when initializing ViltModel: ['classifier.3.bias', 'classifier.3.weight', 'classifier.0.weight', 'classifier.1.bias', 'classifier.0.bias', 'classifier.1.weight']
- This IS expected if you are initializing ViltModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViltModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ViltModel(
  (embeddings): ViltEmbeddings(
    (text_embeddings): TextEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(40, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (patch_embeddings): ViltPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
    )
    (token_type_embeddings): Embedding(2, 768)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViltEncoder(
    (layer): ModuleList(
      (0): ViltLayer(
        (attention): ViltAttention(
          (attention): ViltSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
   

In [5]:
data_df['img_file'] = data_df['img_file'].apply(lambda x: get_img_data(x))
with torch.no_grad():
# img_mode_map = {'P': 0, 'L': 1, 'RGB': 2, 'CMYK': 3, 'RGBA': 4}
    inputs1 = processor(Image.open(data_df['img_file'][0]), data_df['Title'][0] + ' ' + data_df['Alltags'][0], return_tensors='pt', padding=True , truncation=True).to(device)
    outputs1 = model(**inputs1)
    vilt_fea = outputs1.pooler_output
    for index in trange(486193):
        text = data_df['Title'][index + 1] + ', ' + data_df['Alltags'][index + 1]
        try:
            image = Image.open(data_df['img_file'][index + 1])  
            inputs = processor(image, text, return_tensors='pt', padding=True , truncation=True).to(device)
            outputs = model(**inputs)
            vilt_fea1 = outputs.pooler_output
            vilt_fea = torch.concat((vilt_fea,vilt_fea1),0)
        except: 
            image = Image.open('../data/none_picture.jpg')
            inputs = processor(image, text, return_tensors='pt', padding=True , truncation=True).to(device)
            outputs = model(**inputs)
            vilt_fea1 = outputs.pooler_output
            vilt_fea = torch.concat((vilt_fea,vilt_fea1),0)
vilt_fea_768 = pd.DataFrame(vilt_fea.cpu().detach().numpy())
vilt_fea_768.columns = ['viltvqa_fea_{}'.format(i) for i in range(768)]
vilt_fea_768.to_csv('../data/vilt_fea_vqa.csv', header=True, index=None)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 486193/486193 [3:59:21<00:00, 33.85it/s]


AttributeError: 'Tensor' object has no attribute 'to_csv'

In [6]:
vilt_fea_768

,viltvqa_fea_0,viltvqa_fea_1,viltvqa_fea_2,viltvqa_fea_3,viltvqa_fea_4,viltvqa_fea_5,viltvqa_fea_6,viltvqa_fea_7,viltvqa_fea_8,viltvqa_fea_9,...,viltvqa_fea_758,viltvqa_fea_759,viltvqa_fea_760,viltvqa_fea_761,viltvqa_fea_762,viltvqa_fea_763,viltvqa_fea_764,viltvqa_fea_765,viltvqa_fea_766,viltvqa_fea_767
0,-0.814128,-0.694269,-0.234653,-0.484401,-0.096432,0.241817,0.229497,0.055558,0.139545,-0.884845,...,0.315131,0.239216,-0.493226,0.382592,-0.298273,0.482726,-0.329398,0.172889,0.331349,-0.215726
1,-0.715331,-0.751090,-0.049836,-0.223661,0.170536,0.308837,0.719889,-0.379096,-0.386496,-0.432119,...,-0.787129,0.377099,-0.135965,0.218942,0.218627,0.132974,0.399288,-0.266744,0.438168,0.352710
2,-0.788853,-0.550316,0.105480,-0.086979,-0.189308,0.114575,0.472300,0.012797,-0.842469,-0.403882,...,0.174423,0.405909,0.241776,0.191860,-0.393805,0.406829,-0.494684,0.053800,0.301847,-0.274030
3,0.566744,-0.670803,-0.039364,0.315698,0.044863,0.201079,0.605891,0.311271,-0.894812,-0.425816,...,0.133044,-0.114484,0.387913,0.248558,-0.066610,0.157745,-0.294297,0.001803,0.410927,0.153709
4,0.496288,0.191614,0.004553,-0.139520,0.190589,-0.101858,0.394599,-0.047913,-0.412546,-0.231087,...,0.236242,0.466902,-0.222561,-0.061981,-0.021660,0.001125,-0.149710,0.290287,-0.238527,-0.003913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,0.274900,-0.308347,0.305278,0.285758,0.326521,0.002827,0.838962,-0.249448,-0.651783,-0.060371,...,-0.034823,0.254697,-0.346216,-0.202336,-0.147684,0.071354,-0.135466,-0.194680,0.166100,-0.407186
486190,0.498949,-0.167520,-0.230888,0.161650,0.056980,-0.286838,0.502601,0.225557,-0.751049,-0.373558,...,-0.017615,0.306145,-0.109546,-0.002779,0.084461,0.105725,0.233413,0.267870,0.317124,0.061053
486191,-0.286199,-0.391604,-0.200778,-0.070975,-0.299070,0.492843,0.634418,0.047439,-0.353635,-0.112784,...,-0.101830,-0.091368,-0.132160,0.135501,-0.167699,-0.191692,0.251878,0.057899,0.699864,0.174545
486192,-0.523017,-0.468343,-0.568906,-0.071479,-0.068754,0.385218,0.710879,0.538697,0.582647,-0.602718,...,0.146164,-0.001863,0.322281,0.288081,-0.352652,0.434992,0.100914,0.413298,-0.327786,-0.225809


In [7]:
vilt_fea_768.to_csv('../data/vilt_fea_vqa.csv', header=True, index=None)